In [1]:
import sys

sys.path.append(r"C:\Users\user\Desktop\SHIFT\credit_scoring")

In [2]:
from credit_scoring.src.app.utils import postgres_connector
from credit_scoring.src.config import config_file

In [3]:
data_full_path = config_file.DATA_FULL_PATH
db_args = config_file.DB_ARGS

In [4]:
sql_connecter = postgres_connector.PostgreSqlConnector(db_args)

Давайте попробуем использовать коннектер

Импортируем bureau

In [5]:
sql_schema_query = """
    DROP TABLE IF EXISTS bureau;
    CREATE TABLE bureau(
      SK_ID_CURR integer,
      SK_ID_BUREAU integer,
      CREDIT_ACTIVE varchar(13),
      CREDIT_CURRENCY varchar(15),
      DAYS_CREDIT smallint,
      CREDIT_DAY_OVERDUE smallint,
      DAYS_CREDIT_ENDDATE real,
      DAYS_ENDDATE_FACT real,
      AMT_CREDIT_MAX_OVERDUE real,
      CNT_CREDIT_PROLONG smallint,
      AMT_CREDIT_SUM real,
      AMT_CREDIT_SUM_DEBT real,
      AMT_CREDIT_SUM_LIMIT real,
      AMT_CREDIT_SUM_OVERDUE real,
      CREDIT_TYPE varchar(49),
      DAYS_CREDIT_UPDATE integer,
      AMT_ANNUITY  real)
"""

In [6]:
%%time
sql_connecter.send_sql_query(sql_schema_query)

CPU times: total: 0 ns
Wall time: 103 ms


Теперь "зальем" данные.

In [7]:
sql_data_query = f"""
COPY bureau(
  SK_ID_CURR,
  SK_ID_BUREAU,
  CREDIT_ACTIVE,
  CREDIT_CURRENCY,
  DAYS_CREDIT,
  CREDIT_DAY_OVERDUE,
  DAYS_CREDIT_ENDDATE,
  DAYS_ENDDATE_FACT,
  AMT_CREDIT_MAX_OVERDUE,
  CNT_CREDIT_PROLONG,
  AMT_CREDIT_SUM,
  AMT_CREDIT_SUM_DEBT,
  AMT_CREDIT_SUM_LIMIT,
  AMT_CREDIT_SUM_OVERDUE,
  CREDIT_TYPE,
  DAYS_CREDIT_UPDATE,
  AMT_ANNUITY
)
FROM '{data_full_path}bureau.csv' DELIMITER ',' CSV HEADER;
"""

In [8]:
%%time
sql_connecter.send_sql_query(sql_data_query)

CPU times: total: 0 ns
Wall time: 6.01 s


Выведем залитые данные

In [9]:
test_sql = '''
SELECT * FROM bureau
LIMIT 3
'''
sql_connecter.get_df_from_query(test_sql)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,sk_id_bureau,credit_active,credit_currency,days_credit,credit_day_overdue,days_credit_enddate,days_enddate_fact,amt_credit_max_overdue,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_limit,amt_credit_sum_overdue,credit_type,days_credit_update,amt_annuity
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,None,0,91323.0,0.0,None,0.0,Consumer credit,-131,None
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,None,0,225000.0,171342.0,None,0.0,Credit card,-20,None
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,None,0,464323.5,NaN,None,0.0,Consumer credit,-16,None


Создадим объединённую таблицу

In [10]:
sql_schema_query = """
    DROP TABLE IF EXISTS train_test_data;
    CREATE TABLE public.train_test_data(
        SK_ID_CURR integer,
        NAME_CONTRACT_TYPE varchar(15),
        CODE_GENDER varchar(3),
        FLAG_OWN_CAR varchar(1),
        FLAG_OWN_REALTY varchar(1),
        CNT_CHILDREN integer,
        AMT_INCOME_TOTAL double precision,
        AMT_CREDIT double precision,
        AMT_ANNUITY double precision,
        AMT_GOODS_PRICE double precision,
        NAME_TYPE_SUITE varchar(15),
        NAME_INCOME_TYPE varchar(20),
        NAME_EDUCATION_TYPE varchar(29),
        NAME_FAMILY_STATUS varchar(20),
        NAME_HOUSING_TYPE varchar(19),
        REGION_POPULATION_RELATIVE double precision,
        DAYS_BIRTH integer,
        DAYS_EMPLOYED integer,
        DAYS_REGISTRATION double precision,
        DAYS_ID_PUBLISH integer,
        OWN_CAR_AGE real,
        FLAG_MOBIL smallint,
        FLAG_EMP_PHONE smallint,
        FLAG_WORK_PHONE smallint,
        FLAG_CONT_MOBILE smallint,
        FLAG_PHONE smallint,
        FLAG_EMAIL smallint,
        OCCUPATION_TYPE varchar(30),
        CNT_FAM_MEMBERS real,
        REGION_RATING_CLIENT smallint,
        REGION_RATING_CLIENT_W_CITY integer,
        WEEKDAY_APPR_PROCESS_START varchar(9),
        HOUR_APPR_PROCESS_START integer,
        REG_REGION_NOT_LIVE_REGION smallint,
        REG_REGION_NOT_WORK_REGION smallint,
        LIVE_REGION_NOT_WORK_REGION smallint,
        REG_CITY_NOT_LIVE_CITY smallint,
        REG_CITY_NOT_WORK_CITY smallint,
        LIVE_CITY_NOT_WORK_CITY smallint,
        ORGANIZATION_TYPE varchar(33),
        EXT_SOURCE_1 double precision,
        EXT_SOURCE_2 double precision,
        EXT_SOURCE_3 double precision,
        APARTMENTS_AVG double precision,
        BASEMENTAREA_AVG double precision,
        YEARS_BEGINEXPLUATATION_AVG real,
        YEARS_BUILD_AVG double precision,
        COMMONAREA_AVG double precision,
        ELEVATORS_AVG double precision,
        ENTRANCES_AVG double precision,
        FLOORSMAX_AVG real,
        FLOORSMIN_AVG real,
        LANDAREA_AVG double precision,
        LIVINGAPARTMENTS_AVG double precision,
        LIVINGAREA_AVG double precision,
        NONLIVINGAPARTMENTS_AVG double precision,
        NONLIVINGAREA_AVG double precision,
        APARTMENTS_MODE double precision,
        BASEMENTAREA_MODE double precision,
        YEARS_BEGINEXPLUATATION_MODE real,
        YEARS_BUILD_MODE double precision,
        COMMONAREA_MODE double precision,
        ELEVATORS_MODE real,
        ENTRANCES_MODE real,
        FLOORSMAX_MODE real,
        FLOORSMIN_MODE real,
        LANDAREA_MODE double precision,
        LIVINGAPARTMENTS_MODE double precision,
        LIVINGAREA_MODE double precision,
        NONLIVINGAPARTMENTS_MODE double precision,
        NONLIVINGAREA_MODE double precision,
        APARTMENTS_MEDI double precision,
        BASEMENTAREA_MEDI double precision,
        YEARS_BEGINEXPLUATATION_MEDI real,
        YEARS_BUILD_MEDI double precision,
        COMMONAREA_MEDI double precision,
        ELEVATORS_MEDI real,
        ENTRANCES_MEDI real,
        FLOORSMAX_MEDI real,
        FLOORSMIN_MEDI real,
        LANDAREA_MEDI double precision,
        LIVINGAPARTMENTS_MEDI double precision,
        LIVINGAREA_MEDI double precision,
        NONLIVINGAPARTMENTS_MEDI double precision,
        NONLIVINGAREA_MEDI double precision,
        FONDKAPREMONT_MODE varchar(21),
        HOUSETYPE_MODE varchar(16),
        TOTALAREA_MODE double precision,
        WALLSMATERIAL_MODE varchar(12),
        EMERGENCYSTATE_MODE varchar(3),
        OBS_30_CNT_SOCIAL_CIRCLE real,
        DEF_30_CNT_SOCIAL_CIRCLE real,
        OBS_60_CNT_SOCIAL_CIRCLE real,
        DEF_60_CNT_SOCIAL_CIRCLE real,
        DAYS_LAST_PHONE_CHANGE double precision,
        FLAG_DOCUMENT_2 smallint,
        FLAG_DOCUMENT_3 smallint,
        FLAG_DOCUMENT_4 smallint,
        FLAG_DOCUMENT_5 smallint,
        FLAG_DOCUMENT_6 smallint,
        FLAG_DOCUMENT_7 smallint,
        FLAG_DOCUMENT_8 smallint,
        FLAG_DOCUMENT_9 smallint,
        FLAG_DOCUMENT_10 smallint,
        FLAG_DOCUMENT_11 smallint,
        FLAG_DOCUMENT_12 smallint,
        FLAG_DOCUMENT_13 smallint,
        FLAG_DOCUMENT_14 smallint,
        FLAG_DOCUMENT_15 smallint,
        FLAG_DOCUMENT_16 smallint,
        FLAG_DOCUMENT_17 smallint,
        FLAG_DOCUMENT_18 smallint,
        FLAG_DOCUMENT_19 smallint,
        FLAG_DOCUMENT_20 smallint,
        FLAG_DOCUMENT_21 smallint,
        AMT_REQ_CREDIT_BUREAU_HOUR real,
        AMT_REQ_CREDIT_BUREAU_DAY real,
        AMT_REQ_CREDIT_BUREAU_WEEK real,
        AMT_REQ_CREDIT_BUREAU_MON real,
        AMT_REQ_CREDIT_BUREAU_QRT real,
        AMT_REQ_CREDIT_BUREAU_YEAR real);
"""

In [11]:
%%time
sql_connecter.send_sql_query(sql_schema_query)

CPU times: total: 0 ns
Wall time: 76.2 ms


Теперь загрузим объединённые данные

In [12]:
sql_data_query = f"""
COPY train_test_data(
    SK_ID_CURR,
    NAME_CONTRACT_TYPE,
    CODE_GENDER,
    FLAG_OWN_CAR,
    FLAG_OWN_REALTY,
    CNT_CHILDREN,
    AMT_INCOME_TOTAL,
    AMT_CREDIT,
    AMT_ANNUITY,
    AMT_GOODS_PRICE,
    NAME_TYPE_SUITE,
    NAME_INCOME_TYPE,
    NAME_EDUCATION_TYPE,
    NAME_FAMILY_STATUS,
    NAME_HOUSING_TYPE,
    REGION_POPULATION_RELATIVE,
    DAYS_BIRTH,
    DAYS_EMPLOYED,
    DAYS_REGISTRATION,
    DAYS_ID_PUBLISH,
    OWN_CAR_AGE,
    FLAG_MOBIL,
    FLAG_EMP_PHONE,
    FLAG_WORK_PHONE,
    FLAG_CONT_MOBILE,
    FLAG_PHONE,
    FLAG_EMAIL,
    OCCUPATION_TYPE,
    CNT_FAM_MEMBERS,
    REGION_RATING_CLIENT,
    REGION_RATING_CLIENT_W_CITY,
    WEEKDAY_APPR_PROCESS_START,
    HOUR_APPR_PROCESS_START,
    REG_REGION_NOT_LIVE_REGION,
    REG_REGION_NOT_WORK_REGION,
    LIVE_REGION_NOT_WORK_REGION,
    REG_CITY_NOT_LIVE_CITY,
    REG_CITY_NOT_WORK_CITY,
    LIVE_CITY_NOT_WORK_CITY,
    ORGANIZATION_TYPE,
    EXT_SOURCE_1,
    EXT_SOURCE_2,
    EXT_SOURCE_3,
    APARTMENTS_AVG,
    BASEMENTAREA_AVG,
    YEARS_BEGINEXPLUATATION_AVG,
    YEARS_BUILD_AVG,
    COMMONAREA_AVG,
    ELEVATORS_AVG,
    ENTRANCES_AVG,
    FLOORSMAX_AVG,
    FLOORSMIN_AVG,
    LANDAREA_AVG,
    LIVINGAPARTMENTS_AVG,
    LIVINGAREA_AVG,
    NONLIVINGAPARTMENTS_AVG,
    NONLIVINGAREA_AVG,
    APARTMENTS_MODE,
    BASEMENTAREA_MODE,
    YEARS_BEGINEXPLUATATION_MODE,
    YEARS_BUILD_MODE,
    COMMONAREA_MODE,
    ELEVATORS_MODE,
    ENTRANCES_MODE,
    FLOORSMAX_MODE,
    FLOORSMIN_MODE,
    LANDAREA_MODE,
    LIVINGAPARTMENTS_MODE,
    LIVINGAREA_MODE,
    NONLIVINGAPARTMENTS_MODE,
    NONLIVINGAREA_MODE,
    APARTMENTS_MEDI,
    BASEMENTAREA_MEDI,
    YEARS_BEGINEXPLUATATION_MEDI,
    YEARS_BUILD_MEDI,
    COMMONAREA_MEDI,
    ELEVATORS_MEDI,
    ENTRANCES_MEDI,
    FLOORSMAX_MEDI,
    FLOORSMIN_MEDI,
    LANDAREA_MEDI,
    LIVINGAPARTMENTS_MEDI,
    LIVINGAREA_MEDI,
    NONLIVINGAPARTMENTS_MEDI,
    NONLIVINGAREA_MEDI,
    FONDKAPREMONT_MODE,
    HOUSETYPE_MODE,
    TOTALAREA_MODE,
    WALLSMATERIAL_MODE,
    EMERGENCYSTATE_MODE,
    OBS_30_CNT_SOCIAL_CIRCLE,
    DEF_30_CNT_SOCIAL_CIRCLE,
    OBS_60_CNT_SOCIAL_CIRCLE,
    DEF_60_CNT_SOCIAL_CIRCLE,
    DAYS_LAST_PHONE_CHANGE,
    FLAG_DOCUMENT_2,
    FLAG_DOCUMENT_3,
    FLAG_DOCUMENT_4,
    FLAG_DOCUMENT_5,
    FLAG_DOCUMENT_6,
    FLAG_DOCUMENT_7,
    FLAG_DOCUMENT_8,
    FLAG_DOCUMENT_9,
    FLAG_DOCUMENT_10,
    FLAG_DOCUMENT_11,
    FLAG_DOCUMENT_12,
    FLAG_DOCUMENT_13,
    FLAG_DOCUMENT_14,
    FLAG_DOCUMENT_15,
    FLAG_DOCUMENT_16,
    FLAG_DOCUMENT_17,
    FLAG_DOCUMENT_18,
    FLAG_DOCUMENT_19,
    FLAG_DOCUMENT_20,
    FLAG_DOCUMENT_21,
    AMT_REQ_CREDIT_BUREAU_HOUR,
    AMT_REQ_CREDIT_BUREAU_DAY,
    AMT_REQ_CREDIT_BUREAU_WEEK,
    AMT_REQ_CREDIT_BUREAU_MON,
    AMT_REQ_CREDIT_BUREAU_QRT,
    AMT_REQ_CREDIT_BUREAU_YEAR
)
FROM '{data_full_path}train_test_united.csv' DELIMITER ',' CSV HEADER;
"""

In [13]:
%%time
sql_connecter.send_sql_query(sql_data_query)

CPU times: total: 0 ns
Wall time: 7.01 s


Проверяем нормально ли загрузилось

In [14]:
%%time
sql_query = """
    SELECT * FROM train_test_data
    LIMIT 5
"""
sql_connecter.get_df_from_query(sql_query)

CPU times: total: 15.6 ms
Wall time: 85 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,...,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
%%time
sql_query = """
    SELECT * FROM train_test_data
"""
df_train_test_data = sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 12 s
Wall time: 14.6 s


Далее генерируем 5 признаков
Мне в комментариях сказали, что создавать новую колонку и её заполнять не нужно, а просто написать запрос
Так как видео не выложено, то сделаю как мне сказали

Признак №1 флаг аномалий для колонки DAYS_EMPLOYED, где DAYS_EMPLOYED==365243, то True, в противном случае False

In [16]:
%%time
sql_query = '''
SELECT SK_ID_CURR, DAYS_EMPLOYED FROM train_test_data
WHERE DAYS_EMPLOYED=365243
'''
sql_connecter.get_df_from_query(sql_query)


CPU times: total: 31.2 ms
Wall time: 276 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,days_employed
0,100011,365243
1,100015,365243
2,100027,365243
3,100045,365243
4,100050,365243
...,...,...
64643,456008,365243
64644,456119,365243
64645,456123,365243
64646,456169,365243


Признак №2. Отношение кредита к ежемесячному платежу

In [17]:
%%time
sql_query = '''
SELECT SK_ID_CURR, AMT_CREDIT, AMT_ANNUITY, (AMT_CREDIT/AMT_ANNUITY) as NEW_CREDIT_TO_ANNUITY_RATIO
FROM train_test_data
'''
sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 422 ms
Wall time: 860 ms


,sk_id_curr,amt_credit,amt_annuity,new_credit_to_annuity_ratio
0,100002,406597.5,24700.5,16.461104
1,100003,1293502.5,35698.5,36.234085
2,100004,135000.0,6750.0,20.000000
3,100006,312682.5,29686.5,10.532818
4,100007,513000.0,21865.5,23.461618
...,...,...,...,...
356250,456221,412560.0,17473.5,23.610610
356251,456222,622413.0,31909.5,19.505570
356252,456223,315000.0,33205.5,9.486380
356253,456224,450000.0,25128.0,17.908309


Признак №3. Отношение кредита к цене товаров

In [18]:
%%time
sql_query = '''
SELECT SK_ID_CURR, AMT_CREDIT, AMT_GOODS_PRICE, (AMT_CREDIT/AMT_GOODS_PRICE) as NEW_CREDIT_TO_GOODS_RATIO
FROM train_test_data
'''
sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 422 ms
Wall time: 818 ms


,sk_id_curr,amt_credit,amt_goods_price,new_credit_to_goods_ratio
0,100002,406597.5,351000.0,1.158397
1,100003,1293502.5,1129500.0,1.145199
2,100004,135000.0,135000.0,1.000000
3,100006,312682.5,297000.0,1.052803
4,100007,513000.0,513000.0,1.000000
...,...,...,...,...
356250,456221,412560.0,270000.0,1.528000
356251,456222,622413.0,495000.0,1.257400
356252,456223,315000.0,315000.0,1.000000
356253,456224,450000.0,450000.0,1.000000


Признак №4. Количество денег члена семьи (исходя из того, что один взрослый в семье, взял кредит и кормит детей, а второй взрослый не вкладывает в семью, а просто содержит себя)

In [19]:
%%time
sql_query = '''
SELECT SK_ID_CURR, AMT_INCOME_TOTAL, CNT_CHILDREN, (AMT_INCOME_TOTAL/(1+CNT_CHILDREN)) as NEW_INC_PER_CHLD
FROM train_test_data

'''
sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 391 ms
Wall time: 758 ms


,sk_id_curr,amt_income_total,cnt_children,new_inc_per_chld
0,100002,202500.0,0,202500.0
1,100003,270000.0,0,270000.0
2,100004,67500.0,0,67500.0
3,100006,135000.0,0,135000.0
4,100007,121500.0,0,121500.0
...,...,...,...,...
356250,456221,121500.0,0,121500.0
356251,456222,157500.0,2,52500.0
356252,456223,202500.0,1,101250.0
356253,456224,225000.0,0,225000.0


Признак №5. Отношение количества лет машине к дням, которые работает

In [20]:
%%time
sql_query = '''
SELECT SK_ID_CURR, OWN_CAR_AGE, DAYS_EMPLOYED, (OWN_CAR_AGE/DAYS_EMPLOYED) as NEW_CAR_TO_EMPLOY_RATIO
FROM train_test_data
'''
sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 375 ms
Wall time: 744 ms


,sk_id_curr,own_car_age,days_employed,new_car_to_employ_ratio
0,100002,NaN,-637,NaN
1,100003,NaN,-1188,NaN
2,100004,26.0,-225,-0.115556
3,100006,NaN,-3039,NaN
4,100007,NaN,-3038,NaN
...,...,...,...,...
356250,456221,NaN,-5169,NaN
356251,456222,NaN,-1149,NaN
356252,456223,4.0,-3037,-0.001317
356253,456224,NaN,-2731,NaN


# Упражнения

In [21]:
%%time
# 1. вывести средний доход среди всех клиентов
sql_query = '''
SELECT AVG(AMT_INCOME_TOTAL) as average_income
FROM train_test_data
'''
sql_connecter.get_df_from_query(sql_query)

CPU times: total: 0 ns
Wall time: 218 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,average_income
0,170116.05986


In [22]:
%%time
# 2. вывести минимальный и максимальный возраст среди всех клиентов
sql_query = '''
SELECT MIN(-DAYS_BIRTH)/365 as MIN_YEARS_BIRTH, MAX(-DAYS_BIRTH)/365 as MAX_YEARS_BIRTH
FROM train_test_data
'''
sql_connecter.get_df_from_query(sql_query)

CPU times: total: 0 ns
Wall time: 195 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,min_years_birth,max_years_birth
0,20,69


In [23]:
%%time
# 3. вывести количество мужчин и женщин
sql_query = '''
SELECT CODE_GENDER, COUNT(*)
FROM train_test_data
WHERE CODE_GENDER NOT IN ('XNA')
GROUP BY CODE_GENDER
'''
sql_connecter.get_df_from_query(sql_query)

CPU times: total: 0 ns
Wall time: 199 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,code_gender,count
0,F,235126
1,M,121125


In [24]:
%%time
# 4. вывести общую сумму, количество и среднюю сумму, запрошенную клиентами в кредит с авто и без
sql_query = '''
SELECT FLAG_OWN_CAR, SUM(AMT_CREDIT) as SUM_AMT_CREDIT, COUNT(AMT_CREDIT) as COUNT_AMT_CREDIT, AVG(AMT_CREDIT) as AVG_AMT_CREDIT
FROM train_test_data
GROUP BY FLAG_OWN_CAR

'''
sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 0 ns
Wall time: 214 ms


,flag_own_car,sum_amt_credit,count_amt_credit,avg_amt_credit
0,N,1.303948e+11,235235,554317.381867
1,Y,7.900023e+10,121020,652786.569683


In [26]:
%%time
# 5. вывести доли клиентов с различным образованием
sql_query = '''
SELECT NAME_EDUCATION_TYPE, COUNT(NAME_EDUCATION_TYPE)/ (SELECT COUNT(NAME_EDUCATION_TYPE)  FROM train_test_data) :: real as proportion
FROM train_test_data
GROUP BY NAME_EDUCATION_TYPE

'''
sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 15.6 ms
Wall time: 362 ms


,name_education_type,proportion
0,Academic degree,0.000575
1,Higher education,0.245271
2,Incomplete higher,0.033687
3,Lower secondary,0.012045
4,Secondary / secondary special,0.708422


In [61]:
%%time
# 6. Подсчитать количество полных лет для клиентов, у которых есть во владении автомобиль и недвижимость.
# вывести топ 10 по возрастанию

# Проверил, чтобы проверить нужно превратить в вещественное при помощи ::real, например (-DAYS_BIRTH::real/365)

sql_query = '''
SELECT *
FROM (SELECT SK_ID_CURR, (-DAYS_BIRTH/365) as years_old
      FROM train_test_data
      WHERE FLAG_OWN_CAR='Y' AND FLAG_OWN_REALTY='Y'
      ORDER BY DAYS_BIRTH ASC
      LIMIT 10) t1
ORDER BY t1.years_old ASC
'''
sql_connecter.get_df_from_query(sql_query)

CPU times: total: 0 ns
Wall time: 176 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,years_old
0,417899,68
1,311165,68
2,408005,68
3,238603,68
4,381778,68
5,400656,68
6,254900,68
7,175290,68
8,265852,68
9,371965,68


In [27]:
%%time
# 7. вывести тех клиентов, у кого доход на одного члена семьи в два раза больше, чем в среднем на одного члена семьи по выборке
sql_query = '''
SELECT SK_ID_CURR, AMT_INCOME_TOTAL
FROM train_test_data
WHERE (AMT_INCOME_TOTAL/(CNT_FAM_MEMBERS)) > 2 * (
    SELECT AVG(AMT_INCOME_TOTAL/(CNT_FAM_MEMBERS))
    FROM train_test_data)
'''
sql_connecter.get_df_from_query(sql_query)


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 15.6 ms
Wall time: 391 ms


,sk_id_curr,amt_income_total
0,100002,202500.0
1,100033,270000.0
2,100053,202500.0
3,100070,540000.0
4,100079,225000.0
...,...,...
27728,455940,202500.0
27729,455959,225000.0
27730,456009,225000.0
27731,456119,247500.0


In [28]:
%%time
# 8. вывести клиентов старше 60 лет по которым нет данных в bureau
sql_query = '''
SELECT SK_ID_CURR
FROM train_test_data
WHERE (-DAYS_BIRTH/365) > 60 AND  SK_ID_CURR NOT IN (
    SELECT DISTINCT SK_ID_CURR
    FROM bureau)
'''
sql_connecter.get_df_from_query(sql_query)


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 0 ns
Wall time: 1.43 s


,sk_id_curr
0,100118
1,100236
2,100273
3,100287
4,100413
...,...
5051,453372
5052,454405
5053,454418
5054,454612


Для запроса нужна таблица bureau_balance, так что импортируем её

## bureau_balance

In [42]:
%%time
sql_schema_query = """
    DROP TABLE IF EXISTS bureau_balance;
    CREATE TABLE bureau_balance(
      SK_ID_BUREAU integer,
      MONTHS_BALANCE integer,
      STATUS varchar(1)
      )
"""
sql_connecter.send_sql_query(sql_schema_query)

CPU times: total: 15.6 ms
Wall time: 182 ms


In [43]:
%%time
sql_data_query = f"""
COPY bureau_balance(
  SK_ID_BUREAU,
  MONTHS_BALANCE,
  STATUS
  )
FROM '{data_full_path}bureau_balance.csv' DELIMITER ',' CSV HEADER;
"""
sql_connecter.send_sql_query(sql_data_query)

CPU times: total: 0 ns
Wall time: 21.7 s


Проверим работоспособность

In [44]:
%%time
test_sql = '''
SELECT * FROM bureau_balance
LIMIT 3
'''
sql_connecter.get_df_from_query(test_sql)

CPU times: total: 15.6 ms
Wall time: 74.3 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_bureau,months_balance,status
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C


In [58]:
%%time
# 9. вывести женщин, у которых в истории bureau было больше двух кредитов, просроченных на 61 день и более
# отсортировать в порядке убывания по кол-ву таких кредитов
sql_query = '''
SELECT t1.SK_ID_CURR, t1.AMOUNT_OF_OVERDUE_OVER_61_DAY
FROM (
    SELECT SK_ID_CURR, COUNT(SK_ID_BUREAU) as AMOUNT_OF_OVERDUE_OVER_61_DAY
    FROM bureau
    WHERE SK_ID_BUREAU in (
        SELECT DISTINCT SK_ID_BUREAU
        FROM bureau_balance
        WHERE STATUS = '3' or STATUS = '4' or STATUS = '5'
    ) AND SK_ID_CURR in (
        SELECT SK_ID_CURR
        FROM train_test_data
        WHERE CODE_GENDER='F'
        )
    GROUP BY SK_ID_CURR
) t1
WHERE t1.AMOUNT_OF_OVERDUE_OVER_61_DAY > 2
ORDER BY t1.AMOUNT_OF_OVERDUE_OVER_61_DAY DESC
'''
sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 15.6 ms
Wall time: 2.15 s


,sk_id_curr,amount_of_overdue_over_61_day
0,279826,14
1,441227,13
2,246788,13
3,129529,10
4,164862,10
...,...,...
335,184667,3
336,182879,3
337,394391,3
338,272607,3


In [86]:
%%time
# 10. по данным из bureau (БКИ) рассчитать долю просрочки в активных займах для каждого клиента
# вывести топ 7 мужчин с наибольшей суммой просрочки, указав для них, помимо прочего,
# сумму активных кредитов и суммы всех кредитов (активных и закрытых).

# WHERE AMT_CREDIT_SUM <> 0 это значит что у него нет кредитов и он не попадёт в выборку

sql_query = '''
SELECT t1.SK_ID_CURR, t3.SUM_OF_OVERDUE, t1.PROPORTION_OF_OVERDUE, t2.SUM_OF_ACTIVE_CREDITS, t3.SUM_OF_ALL_CREDITS
FROM (
    SELECT SK_ID_CURR, SUM(AMT_CREDIT_SUM_OVERDUE) / SUM(AMT_CREDIT_SUM) AS PROPORTION_OF_OVERDUE
    FROM bureau
    WHERE AMT_CREDIT_SUM <> 0
    GROUP BY SK_ID_CURR
) t1
INNER JOIN  (
    SELECT SK_ID_CURR, SUM(AMT_CREDIT_SUM) AS SUM_OF_ACTIVE_CREDITS
    FROM bureau
    WHERE CREDIT_ACTIVE = 'Active'
    GROUP BY SK_ID_CURR
) t2 ON (t1.SK_ID_CURR=t2.SK_ID_CURR)
INNER JOIN (
    SELECT SK_ID_CURR, SUM(AMT_CREDIT_SUM) AS SUM_OF_ALL_CREDITS, SUM(AMT_CREDIT_SUM_OVERDUE) AS SUM_OF_OVERDUE
    FROM bureau
    GROUP BY SK_ID_CURR
) t3 ON (t2.SK_ID_CURR=t3.SK_ID_CURR)
WHERE t1.SK_ID_CURR IN (
    SELECT SK_ID_CURR
    FROM train_test_data
    WHERE  CODE_GENDER = 'M')
ORDER BY t3.SUM_OF_OVERDUE DESC
LIMIT 7

'''
sql_connecter.get_df_from_query(sql_query)

C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CPU times: total: 0 ns
Wall time: 2.29 s


,sk_id_curr,sum_of_overdue,proportion_of_overdue,sum_of_active_credits,sum_of_all_credits
0,435405,3681063.00,0.908008,3690000.0,4054000.5
1,427996,2901294.00,0.390400,4231615.5,7431583.5
2,394113,1332472.50,0.729802,1530000.0,1825801.0
3,266765,1224474.90,0.861121,1350000.0,1421955.0
4,167085,780192.00,4.422744,158404.5,176404.5
5,154595,742491.00,0.174878,990000.0,4245754.5
6,262411,709669.25,0.349513,1318500.0,1773000.0


# Импорт таблиц оставшихся

## credit_card_balance

In [34]:
%%time
sql_schema_query = """
    DROP TABLE IF EXISTS credit_card_balance;
    CREATE TABLE credit_card_balance(
      SK_ID_PREV integer,
      SK_ID_CURR integer,
      MONTHS_BALANCE integer,
      AMT_BALANCE real,
      AMT_CREDIT_LIMIT_ACTUAL integer,
      AMT_DRAWINGS_ATM_CURRENT real,
      AMT_DRAWINGS_CURRENT real,
      AMT_DRAWINGS_OTHER_CURRENT real,
      AMT_DRAWINGS_POS_CURRENT real,
      AMT_INST_MIN_REGULARITY real,
      AMT_PAYMENT_CURRENT real,
      AMT_PAYMENT_TOTAL_CURRENT real,
      AMT_RECEIVABLE_PRINCIPAL real,
      AMT_RECIVABLE real,
      AMT_TOTAL_RECEIVABLE real,
      CNT_DRAWINGS_ATM_CURRENT real,
      CNT_DRAWINGS_CURRENT smallint,
      CNT_DRAWINGS_OTHER_CURRENT real,
      CNT_DRAWINGS_POS_CURRENT real,
      CNT_INSTALMENT_MATURE_CUM real,
      NAME_CONTRACT_STATUS varchar(13),
      SK_DPD smallint,
      SK_DPD_DEF smallint
      )
"""
sql_connecter.send_sql_query(sql_schema_query)

CPU times: total: 0 ns
Wall time: 179 ms


In [35]:
%%time
sql_data_query = f"""
COPY credit_card_balance(
  SK_ID_PREV,
  SK_ID_CURR,
  MONTHS_BALANCE,
  AMT_BALANCE,
  AMT_CREDIT_LIMIT_ACTUAL,
  AMT_DRAWINGS_ATM_CURRENT,
  AMT_DRAWINGS_CURRENT,
  AMT_DRAWINGS_OTHER_CURRENT,
  AMT_DRAWINGS_POS_CURRENT,
  AMT_INST_MIN_REGULARITY,
  AMT_PAYMENT_CURRENT,
  AMT_PAYMENT_TOTAL_CURRENT,
  AMT_RECEIVABLE_PRINCIPAL,
  AMT_RECIVABLE,
  AMT_TOTAL_RECEIVABLE,
  CNT_DRAWINGS_ATM_CURRENT,
  CNT_DRAWINGS_CURRENT,
  CNT_DRAWINGS_OTHER_CURRENT,
  CNT_DRAWINGS_POS_CURRENT,
  CNT_INSTALMENT_MATURE_CUM,
  NAME_CONTRACT_STATUS,
  SK_DPD,
  SK_DPD_DEF
  )
FROM '{data_full_path}credit_card_balance.csv' DELIMITER ',' CSV HEADER;
"""
sql_connecter.send_sql_query(sql_data_query)

CPU times: total: 0 ns
Wall time: 17.8 s


Проверим

In [36]:
%%time
test_sql = '''
SELECT * FROM credit_card_balance
LIMIT 3
'''
sql_connecter.get_df_from_query(test_sql)

CPU times: total: 15.6 ms
Wall time: 74.5 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_prev,sk_id_curr,months_balance,amt_balance,amt_credit_limit_actual,amt_drawings_atm_current,amt_drawings_current,amt_drawings_other_current,amt_drawings_pos_current,amt_inst_min_regularity,...,amt_recivable,amt_total_receivable,cnt_drawings_atm_current,cnt_drawings_current,cnt_drawings_other_current,cnt_drawings_pos_current,cnt_instalment_mature_cum,name_contract_status,sk_dpd,sk_dpd_def
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.086,31460.086,0.0,0,0.0,0.0,30.0,Active,0,0


## installments_payments

In [37]:
%%time
sql_schema_query = """
    DROP TABLE IF EXISTS installments_payments;
    CREATE TABLE installments_payments(
        SK_ID_PREV  integer,
        SK_ID_CURR  integer,
        NUM_INSTALMENT_VERSION real,
        NUM_INSTALMENT_NUMBER  integer,
        DAYS_INSTALMENT real,
        DAYS_ENTRY_PAYMENT real,
        AMT_INSTALMENT real,
        AMT_PAYMENT real
      )
"""
sql_connecter.send_sql_query(sql_schema_query)

CPU times: total: 15.6 ms
Wall time: 74.8 ms


In [38]:
%%time
sql_data_query = f"""
    COPY installments_payments(
        SK_ID_PREV,
        SK_ID_CURR,
        NUM_INSTALMENT_VERSION,
        NUM_INSTALMENT_NUMBER,
        DAYS_INSTALMENT,
        DAYS_ENTRY_PAYMENT,
        AMT_INSTALMENT,
        AMT_PAYMENT
      )
    FROM '{data_full_path}installments_payments.csv' DELIMITER ',' CSV HEADER;
"""
sql_connecter.send_sql_query(sql_data_query)

CPU times: total: 0 ns
Wall time: 29.7 s


Проверим

In [39]:
%%time
test_sql = '''
    SELECT * FROM installments_payments
    LIMIT 3
'''
sql_connecter.get_df_from_query(test_sql)

CPU times: total: 15.6 ms
Wall time: 71 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_prev,sk_id_curr,num_instalment_version,num_instalment_number,days_instalment,days_entry_payment,amt_instalment,amt_payment
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000


## POS_CASH_balance

In [40]:
%%time
sql_schema_query = """
    DROP TABLE IF EXISTS pos_cash_balance;
    CREATE TABLE pos_cash_balance(
        SK_ID_PREV integer,
        SK_ID_CURR integer,
        MONTHS_BALANCE integer,
        CNT_INSTALMENT real,
        CNT_INSTALMENT_FUTURE real,
        NAME_CONTRACT_STATUS varchar(21),
        SK_DPD integer,
        SK_DPD_DEF integer
      )
"""
sql_connecter.send_sql_query(sql_schema_query)

CPU times: total: 15.6 ms
Wall time: 73 ms


In [41]:
%%time
sql_data_query = f"""
    COPY pos_cash_balance(
        SK_ID_PREV,
        SK_ID_CURR,
        MONTHS_BALANCE,
        CNT_INSTALMENT,
        CNT_INSTALMENT_FUTURE,
        NAME_CONTRACT_STATUS,
        SK_DPD,
        SK_DPD_DEF
      )
    FROM '{data_full_path}POS_CASH_balance.csv' DELIMITER ',' CSV HEADER;
"""
sql_connecter.send_sql_query(sql_data_query)

CPU times: total: 0 ns
Wall time: 19.6 s


In [42]:
%%time
test_sql = '''
    SELECT * FROM pos_cash_balance
    LIMIT 3
'''
sql_connecter.get_df_from_query(test_sql)

CPU times: total: 0 ns
Wall time: 72.8 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_prev,sk_id_curr,months_balance,cnt_instalment,cnt_instalment_future,name_contract_status,sk_dpd,sk_dpd_def
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0


## previous_application

In [49]:
%%time
sql_schema_query = """
    DROP TABLE IF EXISTS previous_application;
    CREATE TABLE previous_application(
        SK_ID_PREV integer,
        SK_ID_CURR integer,
        NAME_CONTRACT_TYPE varchar(15),
        AMT_ANNUITY real,
        AMT_APPLICATION real,
        AMT_CREDIT real,
        AMT_DOWN_PAYMENT real,
        AMT_GOODS_PRICE real,
        WEEKDAY_APPR_PROCESS_START varchar(9),
        HOUR_APPR_PROCESS_START integer,
        FLAG_LAST_APPL_PER_CONTRACT varchar(1),
        NFLAG_LAST_APPL_IN_DAY integer,
        RATE_DOWN_PAYMENT real,
        RATE_INTEREST_PRIMARY real,
        RATE_INTEREST_PRIVILEGED real,
        NAME_CASH_LOAN_PURPOSE varchar(48),
        NAME_CONTRACT_STATUS varchar(12),
        DAYS_DECISION integer,
        NAME_PAYMENT_TYPE varchar(41),
        CODE_REJECT_REASON varchar(6),
        NAME_TYPE_SUITE varchar(15),
        NAME_CLIENT_TYPE varchar(9),
        NAME_GOODS_CATEGORY varchar(24),
        NAME_PORTFOLIO varchar(5),
        NAME_PRODUCT_TYPE varchar(7),
        CHANNEL_TYPE varchar(26),
        SELLERPLACE_AREA integer,
        NAME_SELLER_INDUSTRY varchar(20),
        CNT_PAYMENT real,
        NAME_YIELD_GROUP varchar(10),
        PRODUCT_COMBINATION varchar(30),
        DAYS_FIRST_DRAWING real,
        DAYS_FIRST_DUE real,
        DAYS_LAST_DUE_1ST_VERSION real,
        DAYS_LAST_DUE real,
        DAYS_TERMINATION real,
        NFLAG_INSURED_ON_APPROVAL real
      )
"""
sql_connecter.send_sql_query(sql_schema_query)

CPU times: total: 15.6 ms
Wall time: 70.7 ms


In [50]:
%%time
sql_data_query = f"""
    COPY previous_application(
        SK_ID_PREV,
        SK_ID_CURR,
        NAME_CONTRACT_TYPE,
        AMT_ANNUITY,
        AMT_APPLICATION,
        AMT_CREDIT,
        AMT_DOWN_PAYMENT,
        AMT_GOODS_PRICE,
        WEEKDAY_APPR_PROCESS_START,
        HOUR_APPR_PROCESS_START,
        FLAG_LAST_APPL_PER_CONTRACT,
        NFLAG_LAST_APPL_IN_DAY,
        RATE_DOWN_PAYMENT,
        RATE_INTEREST_PRIMARY,
        RATE_INTEREST_PRIVILEGED,
        NAME_CASH_LOAN_PURPOSE,
        NAME_CONTRACT_STATUS,
        DAYS_DECISION,
        NAME_PAYMENT_TYPE,
        CODE_REJECT_REASON,
        NAME_TYPE_SUITE,
        NAME_CLIENT_TYPE,
        NAME_GOODS_CATEGORY,
        NAME_PORTFOLIO,
        NAME_PRODUCT_TYPE,
        CHANNEL_TYPE,
        SELLERPLACE_AREA,
        NAME_SELLER_INDUSTRY,
        CNT_PAYMENT,
        NAME_YIELD_GROUP,
        PRODUCT_COMBINATION,
        DAYS_FIRST_DRAWING,
        DAYS_FIRST_DUE,
        DAYS_LAST_DUE_1ST_VERSION,
        DAYS_LAST_DUE,
        DAYS_TERMINATION,
        NFLAG_INSURED_ON_APPROVAL
      )
    FROM '{data_full_path}previous_application.csv' DELIMITER ',' CSV HEADER;
"""
sql_connecter.send_sql_query(sql_data_query)

CPU times: total: 0 ns
Wall time: 11.3 s


In [51]:
%%time
test_sql = '''
    SELECT * FROM previous_application
    LIMIT 3
'''
sql_connecter.get_df_from_query(test_sql)

CPU times: total: 0 ns
Wall time: 75.5 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_prev,sk_id_curr,name_contract_type,amt_annuity,amt_application,amt_credit,amt_down_payment,amt_goods_price,weekday_appr_process_start,hour_appr_process_start,...,name_seller_industry,cnt_payment,name_yield_group,product_combination,days_first_drawing,days_first_due,days_last_due_1st_version,days_last_due,days_termination,nflag_insured_on_approval
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0


## sample_submission

In [46]:
%%time
sql_schema_query = """
    DROP TABLE IF EXISTS sample_submission;
    CREATE TABLE sample_submission(
        SK_ID_CURR integer,
        TARGET real
      )
"""
sql_connecter.send_sql_query(sql_schema_query)

CPU times: total: 0 ns
Wall time: 80.5 ms


In [47]:
%%time
sql_data_query = f"""
    COPY sample_submission(
        SK_ID_CURR,
        TARGET
      )
    FROM '{data_full_path}sample_submission.csv' DELIMITER ',' CSV HEADER;
"""
sql_connecter.send_sql_query(sql_data_query)

CPU times: total: 15.6 ms
Wall time: 104 ms


In [48]:
%%time
test_sql = '''
    SELECT * FROM sample_submission
    LIMIT 3
'''
sql_connecter.get_df_from_query(test_sql)

CPU times: total: 31.2 ms
Wall time: 65.5 ms


C:\Users\user\Desktop\SHIFT\credit_scoring\src\app\utils\postgres_connector.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,target
0,100001,0.5
1,100005,0.5
2,100013,0.5
